# Obligatorisk innlevering 2 - oppgave 2

I denne oppgaven skal vi forsøke å forutsi oppsigelser ved en arbeidsplass basert på en rekke variabler som f.eks alder, kjønn, utdannelse, lønn, osv.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score, accuracy_score, confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from scipy.stats import ttest_ind
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

In [2]:
# I denne oppgaven skal dere bruke følgende datasett:
df = pd.read_csv("./datasett2.csv")

### Preprosessering:
For å forenkle håndteringen av kategoriske data har jeg her skrevet noe kode som gjør om strenger til intergers

Følgende kolonner er formatert som *object* og inneholder kategoriske variabler i form av strenger

In [3]:
kolonner_obj = df.select_dtypes(include=['object']).columns
print(kolonner_obj)

Index(['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole',
       'MaritalStatus', 'Over18', 'OverTime'],
      dtype='object')


I koden under omformer jeg strengene til integers og gir dere "nøkkelen" til å konvertere det tilbake igjen

In [4]:
for i in kolonner_obj:
    print("-----------")
    print(i)
    print("-----------")
    for num, j in enumerate(list(df[i].unique())):
        print("{} = {}".format(j,num))
        df[i].loc[df[i]==j] = num
    df[i] = pd.to_numeric(df[i])

-----------
BusinessTravel
-----------
Travel_Frequently = 0
Travel_Rarely = 1
Non-Travel = 2
-----------
Department
-----------
Research & Development = 0
Sales = 1
Human Resources = 2
-----------
EducationField
-----------
Medical = 0
Other = 1
Marketing = 2
Life Sciences = 3
Technical Degree = 4
Human Resources = 5
-----------
Gender
-----------
Male = 0
Female = 1
-----------
JobRole
-----------
Laboratory Technician = 0
Sales Representative = 1
Sales Executive = 2
Healthcare Representative = 3
Manager = 4
Manufacturing Director = 5
Research Scientist = 6
Human Resources = 7
Research Director = 8
-----------
MaritalStatus
-----------
Married = 0
Divorced = 1
Single = 2
-----------
Over18
-----------
Y = 0
-----------
OverTime
-----------
Yes = 0
No = 1


C:\Users\Bjorn\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


I outputen fra koden nedenfor viser datatypen til de forskjellige kolonnene, som nå etter preprosesseringen bare består av integers (int64). 

In [5]:
print(df.dtypes)

id                          int64
Age                         int64
BusinessTravel              int64
DailyRate                   int64
Department                  int64
DistanceFromHome            int64
Education                   int64
EducationField              int64
EmployeeCount               int64
EnvironmentSatisfaction     int64
Gender                      int64
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
JobRole                     int64
JobSatisfaction             int64
MaritalStatus               int64
MonthlyIncome               int64
MonthlyRate                 int64
NumCompaniesWorked          int64
Over18                      int64
OverTime                    int64
PercentSalaryHike           int64
PerformanceRating           int64
RelationshipSatisfaction    int64
StandardHours               int64
StockOptionLevel            int64
TotalWorkingYears           int64
TrainingTimesLastYear       int64
WorkLifeBalanc

In [6]:
# Her deler jeg datasettet inn i egenskaper (X) og target (y) (fjerner også den første kolonnen siden det bare er en ID 
# og har dermed ingen prediktiv verdi)

X = df.iloc[:,1:-1]
y = df.iloc[:,-1]

# Tilslutt deler jeg inn i et trening- og et testsett (75%/25%)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=0)

# PS: Ikke endre denne kodecellen!

#### Oppgave 2.1) 
Visualiser $y\_train$ og $y\_val$ i et histogram. Kan du, basert på figuren, se hva slags problem vi skal løse (regresjon/klassifisering)? Begrunn svaret

In [7]:
# Identifiser og konverter kategoriske data til numeriske
kolonner_obj = df.select_dtypes(include=['object']).columns
for col in kolonner_obj:
    df[col] = pd.factorize(df[col])[0]

# Splitt data i trenings- og testsett
X = df.iloc[:, 1:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Visualiser y_train og y_test
plt.figure(figsize=(12, 6))

# Histogram for y_train
plt.subplot(1, 2, 1)
plt.hist(y_train, bins=10, edgecolor='k', alpha=0.7)
plt.title('Histogram of y_train')
plt.xlabel('Target Values')
plt.ylabel('Frequency')

# Histogram for y_test
plt.subplot(1, 2, 2)
plt.hist(y_test, bins=10, edgecolor='k', alpha=0.7)
plt.title('Histogram of y_test')
plt.xlabel('Target Values')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

**Svar:**

*Histogrammene viser fordelingen av måldata i henholdsvis trenings- og testsettet. Begge histogrammene viser to tydelige verdier (0 og 1), som bekrefter at dette er en klassifikasjonsoppgave.*

#### Oppgave 2.2)
Velg to modeller og gjør repeterte kryssvalideringer $(10)$ med $K=5$
* Velg hvorvidt du vil bruke "default" hyperparamtre for modellene eller om du vil gjøre et hyperparametersøk
* Skår ytelsen til modellene ved hjelp av F1-score og accuracy
* Visualiser F1-skårene fra de repeterte kryssvalideringene som et boksplot
* Undersøk om modell 1 er bedre eller dårligere enn modell 2 ved å gjøre en to-sidig T-test på F1-skårene fra de repeterte kryssvalideringene 
* Tolk resultatene skriftlig til slutt 


In [8]:
# Skaler dataene
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definer modellene
model1 = LogisticRegression(random_state=0, max_iter=5000)
model2 = RandomForestClassifier(random_state=0)

# Kryssvalidering
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=0)
f1_scorer = make_scorer(f1_score)

# Utfør kryssvalidering for begge modellene
f1_scores_model1 = cross_val_score(model1, X_scaled, y, scoring=f1_scorer, cv=cv)
f1_scores_model2 = cross_val_score(model2, X_scaled, y, scoring=f1_scorer, cv=cv)

# Visualiser F1-scorer som boksplot
plt.figure(figsize=(10, 6))
plt.boxplot([f1_scores_model1, f1_scores_model2], labels=["Logistic Regression", "Random Forest"])
plt.title("F1 Scores from Repeated Cross-Validation")
plt.ylabel("F1 Score")
plt.show()

# T-test for F1-scorer
t_stat, p_value = ttest_ind(f1_scores_model1, f1_scores_model2)

# Oppsummer resultater
results_summary = {
    "Model": ["Logistic Regression", "Random Forest"],
    "F1 Mean": [np.mean(f1_scores_model1), np.mean(f1_scores_model2)],
    "F1 Std": [np.std(f1_scores_model1), np.std(f1_scores_model2)],
    "T-Statistic": [t_stat, None],
    "P-Value": [p_value, None]
}

results_df = pd.DataFrame(results_summary)

# Skriv ut resultater
print("Resultater fra oppgave 2.2:")
print(results_df)

**Svar:**

*Logistic Regression har en betydelig høyere F1-score enn Random Forest uten bruk av SMOTE, basert på kryssvalideringen:

- Logistic Regression: Gjennomsnittlig F1-score = 0.316 (Standardavvik = 0.066)
- Random Forest: Gjennomsnittlig F1-score = 0.135 (Standardavvik = 0.058)
- T-testen gir en t-verdi = 14.387 og en svært lav p-verdi (6.72e-26), noe som indikerer at forskjellen mellom modellene er statistisk signifikant.

Dette betyr at Logistic Regression presterer bedre enn Random Forest i denne oppgaven, uten justering for klasseubalanse.*

#### Oppgave 2.3) 
Tren de samme modellene men denne gangen med et balansert datasett
* Bruk imbalaced learn og SMOTE til å oppsample minioritetsklassen (bare oppsample treningsdataene i kryssvalideringen - ikke valideringsdataene)
* Skår ytelsen til modellene ved hjelp av F1-score og accuracy
* Visualiser F1-skårene fra de repeterte kryssvalideringene som et boksplot
* Undersøk om modell 1 er bedre eller dårligere enn modell 2 ved å gjøre en to-sidig T-test på F1-skårene fra de repeterte kryssvalideringene 
* Tolk resultatene og sammenlign resultatene med 2.2) 

In [9]:
# Definer SMOTE
smote = SMOTE(random_state=0)

# Kryssvalidering
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Lagring av F1-scorer
f1_scores_model1_smote = []
f1_scores_model2_smote = []

# Manuell implementering av SMOTE i kryssvalidering
for train_idx, test_idx in cv.split(X_scaled, y):
    # Splitt data
    X_train_fold, X_test_fold = X_scaled[train_idx], X_scaled[test_idx]
    y_train_fold, y_test_fold = y[train_idx], y[test_idx]
    
    # Bruk SMOTE på treningsdata
    X_train_smote, y_train_smote = smote.fit_resample(X_train_fold, y_train_fold)
    
    # Logistic Regression
    model1.fit(X_train_smote, y_train_smote)
    y_pred_model1 = model1.predict(X_test_fold)
    f1_scores_model1_smote.append(f1_score(y_test_fold, y_pred_model1))
    
    # Random Forest
    model2.fit(X_train_smote, y_train_smote)
    y_pred_model2 = model2.predict(X_test_fold)
    f1_scores_model2_smote.append(f1_score(y_test_fold, y_pred_model2))

# Visualiser F1-scorer med SMOTE som boksplot
plt.figure(figsize=(10, 6))
plt.boxplot([f1_scores_model1_smote, f1_scores_model2_smote], labels=["Logistic Regression w/ SMOTE", "Random Forest w/ SMOTE"])
plt.title("F1 Scores with SMOTE")
plt.ylabel("F1 Score")
plt.show()

# T-test for F1-scorer med SMOTE
t_stat_smote, p_value_smote = ttest_ind(f1_scores_model1_smote, f1_scores_model2_smote)

# Oppsummer resultater
results_summary_smote = {
    "Model": ["Logistic Regression w/ SMOTE", "Random Forest w/ SMOTE"],
    "F1 Mean": [np.mean(f1_scores_model1_smote), np.mean(f1_scores_model2_smote)],
    "F1 Std": [np.std(f1_scores_model1_smote), np.std(f1_scores_model2_smote)],
    "T-Statistic": [t_stat_smote, None],
    "P-Value": [p_value_smote, None]
}

results_df_smote = pd.DataFrame(results_summary_smote)

# Skriv ut resultater
print("Resultater fra oppgave 2.3:")
print(results_df_smote)

**Svar:**

*Etter bruk av SMOTE, forbedres F1-scoren for begge modellene, spesielt for Logistic Regression.
Forskjellen mellom modellene er mindre signifikant (p-verdi = 0.094753), noe som tyder på at SMOTE jevner ut modellene.*

#### Oppgave 2.4)
Tren den beste modellen på nytt og utfør prediksjon på testsettet. Mål nå ytelsen på testsettet ved bruk av sensitivitet, spesifisitet og presisjon, itillegg til F1-skår og accuracy

In [10]:
# Tren den beste modellen (Random Forest w/ SMOTE)
best_model = RandomForestClassifier(random_state=0)
best_model.fit(X_train, y_train)

# Prediksjoner på testsettet
y_pred = best_model.predict(X_test)

# Beregn ytelsesmål
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)  # Recall
specificity = tn / (tn + fp)
precision = precision_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Vis resultater
print("Resultater fra oppgave 2.4:")
print(f"Sensitivity (Recall): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Accuracy: {accuracy:.3f}")
print(f"F1 Score: {f1:.3f}")

#### Oppgave 2.5)
Del opp testsettet basert på kjønn og gjør en analyse av sensitivitet, spesifisitet og presisjon for menn og det samme for kvinner. Skriv ut og diskuter resultatet og forsøk å svar på følgende spørsmål:
* Er modellen like god for begge kjønn? 
* Hvis ikke, hva tror du er årsaken og hvilke tiltak kan vi gjøre for å forbedre modellen? 

In [11]:
if 'Gender' in df.columns:
    # Del opp testsettet basert på kjønn
    X_test_men = X_test[df.loc[X_test.index, 'Gender'] == 1]
    y_test_men = y_test[df.loc[X_test.index, 'Gender'] == 1]
    
    X_test_women = X_test[df.loc[X_test.index, 'Gender'] == 0]
    y_test_women = y_test[df.loc[X_test.index, 'Gender'] == 0]
    
    # Prediksjoner for menn
    y_pred_men = best_model.predict(X_test_men)
    conf_matrix_men = confusion_matrix(y_test_men, y_pred_men)
    tn_men, fp_men, fn_men, tp_men = conf_matrix_men.ravel()
    sensitivity_men = tp_men / (tp_men + fn_men) if (tp_men + fn_men) > 0 else 0
    specificity_men = tn_men / (tn_men + fp_men) if (tn_men + fp_men) > 0 else 0
    precision_men = precision_score(y_test_men, y_pred_men) if (tp_men + fp_men) > 0 else 0
    
    # Prediksjoner for kvinner
    y_pred_women = best_model.predict(X_test_women)
    conf_matrix_women = confusion_matrix(y_test_women, y_pred_women)
    tn_women, fp_women, fn_women, tp_women = conf_matrix_women.ravel()
    sensitivity_women = tp_women / (tp_women + fn_women) if (tp_women + fn_women) > 0 else 0
    specificity_women = tn_women / (tn_women + fp_women) if (tn_women + fp_women) > 0 else 0
    precision_women = precision_score(y_test_women, y_pred_women) if (tp_women + fp_women) > 0 else 0
    
    # Vis resultater
    print("Resultater for menn:")
    print(f"Sensitivity (Recall): {sensitivity_men:.3f}")
    print(f"Specificity: {specificity_men:.3f}")
    print(f"Precision: {precision_men:.3f}")
    
    print("\nResultater for kvinner:")
    print(f"Sensitivity (Recall): {sensitivity_women:.3f}")
    print(f"Specificity: {specificity_women:.3f}")
    print(f"Precision: {precision_women:.3f}")
    
    # Diskusjon
    print("\nDiskusjon:")
    if sensitivity_men != sensitivity_women or specificity_men != specificity_women or precision_men != precision_women:
        print("Modellen presterer ulikt for menn og kvinner. Dette kan skyldes:")
        print("- Ubalanse i datasettet mellom kjønnene.")
        print("- At kjønnsvariabelen påvirker modellens evne til å lære riktig mønster.")
        print("\nTiltak for forbedring inkluderer:")
        print("- Bruke balanserte datasett for trening.")
        print("- Utføre separate treningsmodeller for hvert kjønn.")
        print("- Analysere ytterligere feilkilder i dataene.")
else:
    print("Datasettet mangler en 'Gender'-kolonne. Kan ikke utføre kjønnsbasert analyse.")

Kjør denne cellen til slutt og legg ved den genererte requirements.txt fila i pull requestet

In [12]:
#!pip freeze > requirements.txt